In [66]:
import pandas as pd
import pandas as pd
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import numpy as np
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sklearn.metrics import mean_squared_error
from sktime.forecasting.compose import make_reduction
from sklearn.exceptions import ConvergenceWarning
import requests
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.linear_model import ElasticNetCV
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    ExpandingWindowSplitter,
)
from sktime.forecasting.compose import MultiplexForecaster
from sklearn.neighbors import KNeighborsRegressor
from sktime.forecasting.ets import AutoETS
from sktime.transformations.series.boxcox import LogTransformer


import warnings

data_df = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/supply_load_price.csv",
    parse_dates=["Date (MST)"],
    index_col="Date (MST)",
)

data_df = data_df.sort_index()
# select dates after dec 1 2022
data_df = data_df.loc["2022-06-01":]

In [67]:
X = data_df.drop(columns=["price"])
y = data_df["price"]

In [68]:
window = 24
X['rolling_mean'] = y.rolling(window).mean().rolling(2).mean().shift(-window // 2)
X['rolling_std'] = y.rolling(window).std().rolling(2).mean().shift(-window // 2)
X['rolling_min'] = y.rolling(window).min().rolling(2).mean().shift(-window // 2)
X['rolling_max'] = y.rolling(window).max().rolling(2).mean().shift(-window // 2)
X['rolling_median'] = y.rolling(window).median().rolling(2).mean().shift(-window // 2)

In [69]:
X['exp_moving_avg'] = y.ewm(span=24).mean()

In [70]:
X = X.dropna()
y = y[X.index]

In [71]:
X[X['exp_moving_avg'] > 1000].any().any()

False

In [72]:
X[X['rolling_mean'] > 1000].any().any() and X[X['rolling_std'] > 1000].any().any() and X[X['rolling_min'] > 1000].any().any() and X[X['rolling_max'] > 1000].any().any() and X[X['rolling_median'] > 1000].any().any()

False

In [73]:
y[y>1000].any()

False

In [74]:
len(X)

7271

In [75]:
from sktime.transformations.series.feature_selection import FeatureSelection

transformer = FeatureSelection(method="feature-importances", n_columns=20)

transformer.fit_transform(X, y).head(2)

c:\Users\rkris\miniconda3\envs\slalomenv\Lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,relative_gas_reserve,hydro_tng,exp_moving_avg,hydro_reserve_margin,rolling_max,other_reserve_margin,hydro_supply_mix,gas_tng,rolling_std,load_on_gas_reserve,other_tng,wind_avail,rolling_mean,rolling_median,rolling_min,storage_tng,gas_reserve_margin,hydro_avail,gas_tng_ratio,storage_reserve_margin
Date (MST),,,,,,,,,,,,,,,,,,,,
2022-06-01 12:00:00,0.240158,272.462938,290.455414,0.616249,559.57,-0.589333,0.030438,6550.406149,144.189217,0.059718,208.202558,2176.0,242.081875,198.135,94.17,0.0,0.058850,710.0,0.731767,1.0
2022-06-01 13:00:00,0.245573,222.685837,281.892738,0.688115,559.57,-0.539988,0.024989,6526.300993,143.425073,0.062129,201.738381,2187.0,243.216250,198.135,93.60,0.0,0.061099,714.0,0.732349,1.0


In [76]:
from sklearn.preprocessing import StandardScaler
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor

pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        # ("log_transformer", LogTransformer()),
                        # ("deseasonalizer_weekly", Deseasonalizer(sp=24*7, model="additive")),
                        (
                            "forecast",
                            make_reduction(
                                ElasticNetCV(n_jobs=-1),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

In [77]:
# pipe.fit(y, X, fh=np.arange(1, 13))

In [78]:
import pandas as pd

def create_lagged_columns(X, lag_range=24):
    lagged_names = []

    for col in X:
        for lag in range(1, lag_range + 1):
            lagged_names.append(f"{col}_lag{lag}")

    return lagged_names

labels = create_lagged_columns(['price'] + X.columns.values.tolist(), lag_range=24)

In [79]:
len(labels)

1344

In [80]:
# pd.DataFrame({'Coefficient': pipe.forecaster_.forecaster_.estimators_[0].coef_, 'Label': labels}).sort_values(by=['Coefficient'], ascending=False).head(50)

In [82]:
selected_features = [
    "ail",
    "gas_price",
    "gas_tng",
    "coal_tng",
    "wind_tng",
    "gas_avail",
    "wind_avail",
    "gas_reserve_margin",
    "coal_reserve_margin",
    "wind_reserve_margin",
    "other_reserve_margin",
    "gas_supply_mix",
    "coal_supply_mix",
    "wind_supply_mix",
    "other_supply_mix",
    "total_reserve_margin",
    "demand_supply_ratio",
    "fossil_fuel_ratio",
    "rolling_mean",
    "rolling_std",
    "rolling_min",
    "rolling_max",
    "rolling_median",
    "exp_moving_avg",
]

In [83]:
X = X.loc[
    "2023-01-01":"2023-01-31",
    selected_features,
]
y = y.loc["2023-01-01":"2023-01-31"]

In [84]:
cv = ExpandingWindowSplitter(
    initial_window=int(len(X) * 0.94), step_length=1, fh=np.arange(1, 13)
)

n_splits = cv.get_n_splits(y)
print(f"Number of Folds = {n_splits}")

Number of Folds = 34


In [85]:
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredError

list_models = ["elasticnet_pipeline"]

rmse_cv_results = []
rmse_cv_std = []
for i in list_models:
    print(i)
    results = evaluate(
        forecaster=pipe,
        y=y,
        X=X,
        cv=cv,
        strategy="refit",
        return_data=True,
        scoring=MeanSquaredError(square_root=True),
        backend="loky",
        error_score='raise'
    )
    
    rmse = results["test_MeanSquaredError"].mean()
    rmse_std = results["test_MeanSquaredError"].std()
    rmse_cv_results.append(rmse)
    rmse_cv_std.append(rmse_std)

elasticnet_pipeline


In [86]:
rmse_cv_results_df = pd.DataFrame(
    {"Model": list_models, "RMSE_CV": rmse_cv_results, "RMSE_CV_STD": rmse_cv_std}
).sort_values(by=["RMSE_CV"])
rmse_cv_results_df

,Model,RMSE_CV,RMSE_CV_STD
0,elasticnet_pipeline,41.609048,14.023331


In [88]:
re_pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        ("log_transformer", LogTransformer()),
                        # ("deseasonalizer_weekly", Deseasonalizer(sp=24*7, model="additive")),
                        (
                            "forecast",
                            make_reduction(
                                RandomForestRegressor(n_estimators=100, n_jobs=-1),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

In [90]:
re_pipe.fit(y, X, fh=np.arange(1, 13))

ForecastingPipeline(steps=[('standardize',
                            TabularToSeriesAdaptor(transformer=StandardScaler())),
                           ('forecaster',
                            TransformedTargetForecaster(steps=[('log_transformer',
                                                                LogTransformer()),
                                                               ('forecast',
                                                                DirectTabularRegressionForecaster(estimator=RandomForestRegressor(n_jobs=-1),
                                                                                                  window_length=24))]))])

In [91]:
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredError

list_models = ["elasticnet_pipeline"]

rmse_cv_results = []
rmse_cv_std = []
for i in list_models:
    print(i)
    results = evaluate(
        forecaster=re_pipe,
        y=y,
        X=X,
        cv=cv,
        strategy="refit",
        return_data=True,
        scoring=MeanSquaredError(square_root=True),
        backend="loky",
        error_score='raise'
    )
    
    rmse = results["test_MeanSquaredError"].mean()
    rmse_std = results["test_MeanSquaredError"].std()
    rmse_cv_results.append(rmse)
    rmse_cv_std.append(rmse_std)

elasticnet_pipeline


In [92]:
rmse_cv_results_df = pd.DataFrame(
    {"Model": list_models, "RMSE_CV": rmse_cv_results, "RMSE_CV_STD": rmse_cv_std}
).sort_values(by=["RMSE_CV"])
rmse_cv_results_df

,Model,RMSE_CV,RMSE_CV_STD
0,elasticnet_pipeline,38.056256,20.215129


In [94]:
re_pipe.get_params() 

{'steps': [('standardize',
   TabularToSeriesAdaptor(transformer=StandardScaler())),
  ('forecaster',
   TransformedTargetForecaster(steps=[('log_transformer', LogTransformer()),
                                      ('forecast',
                                       DirectTabularRegressionForecaster(estimator=RandomForestRegressor(n_jobs=-1),
                                                                         window_length=24))]))],
 'standardize': TabularToSeriesAdaptor(transformer=StandardScaler()),
 'forecaster': TransformedTargetForecaster(steps=[('log_transformer', LogTransformer()),
                                    ('forecast',
                                     DirectTabularRegressionForecaster(estimator=RandomForestRegressor(n_jobs=-1),
                                                                       window_length=24))]),
 'standardize__fit_in_transform': False,
 'standardize__transformer': StandardScaler(),
 'standardize__transformer__copy': True,
 'standardi